In [1]:
import scipy
from pathlib import Path
import pathlib
from scipy.io.wavfile import read
from scipy.signal import hann
from scipy import signal
from scipy.fftpack import rfft
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from yaafelib import *
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")

cols=['track_name','bpm','harmonic_rythmic_ratio',\
'spectral_centroid','spectral_flatness','count_delta_above_mean',\
'ratio_above_rmse_mean','max_rmse','mean_rms','mean_zcr','liked']
df = pd.DataFrame(columns=cols)

In [2]:
def set_output(artist):
 
    path = pathlib.Path("~/Music/" + artist + "/*.mp3").expanduser()
    parts = path.parts[1:] if path.is_absolute() else path.parts
    return pathlib.Path(path.root).glob(str(pathlib.Path("").joinpath(*parts)))

def count_rmse_above_mean(rmse):
    lower_count = 0
    upper_count = 0
    for e in rmse:
        if e > np.mean(rmse): #arbitrarily chosen, will tune later
            upper_count+=1
        else:
            lower_count +=1
    return upper_count/lower_count

def count_delta_above_mean(rmse):
    dyn_change = 0
    for i in range(len(rmse)-3):
        diff = rmse[i+3]-rmse[i]
        if diff > np.mean(rmse):
            dyn_change += 1
    return dyn_change  

In [3]:
def harmonic_ratio(signal):
    ratios = []
    y_harmonic, y_percussive = librosa.effects.hpss(signal)
    if len(y_harmonic)== len(y_percussive):
        for i in range(len(y_harmonic)):
            if y_harmonic[i] > 0 and y_percussive[i] > 0: #tgety 
                ratios.append(y_harmonic[i]/y_percussive[i])
    else:
        print('lenghts not idenitcal')        
    return np.median(ratios)
    
def extract_features(signal):

    return [
        np.mean(librosa.feature.zero_crossing_rate(signal)),
        np.mean(librosa.feature.rmse(signal)[0]),
        np.max(librosa.feature.rmse(signal)[0]),
        count_rmse_above_mean(librosa.feature.rmse(signal)[0]),
        count_delta_above_mean(librosa.feature.rmse(signal)[0]),
        np.mean(librosa.feature.spectral_flatness(signal)),
        np.mean(librosa.feature.spectral_centroid(signal)),
        
    ]
def other_features(X,sample_rate): # will do RNN after I do SVM, LR, and DT
    cqt = np.mean(librosa.amplitude_to_db(librosa.cqt(y, sr=sr), ref=np.max).T,axis=axis)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    rmse = np.mean(librosa.feature.rmse(S=stft,sr=sample_rate).T,axis=0)
    return cqt,mfccs,chroma,mel,contrast,tonnetz,rmse    
    
 

In [4]:
%%time
# This takes way to long, consider compiling in C or parallel processing
# or some other method

artist = 'Metallica'
output = set_output(artist)

for p in output: 
    track_name = str(p).replace('/Users/mcassettix/Music/' + artist + '/','')
    print(track_name)
    try:
        y_total, sr = librosa.load(p) 
    except:
        print('loading issue, skipping track',p)
        continue
    
    #Sample each track multiple times to increase samples
    #Take 30 second slices of a song, rounded down 
    times = librosa.frames_to_time(np.arange(len(y_total)))
    if  times[-1] > 30: #standarizing length to 2 minutes
        max_time = len(y_total)/sr
        num_slices = math.floor(max_time/30) 
        hop_length = num_slices * sr
        max_x = num_slices*30*sr        
    else:
        print('Warning: you need pick a time range')
        print('skipping track ',p)
        continue 
    frames = list(range(hop_length,max_x, hop_length))
 
    for index in range(len(frames)-1):
        y = y_total[frames[index]:frames[index + 1]]
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        #print('Estimated tempo: {:.2f} beats per minute'.format(tempo))
        features = extract_features(y)
        features.append(harmonic_ratio(y))
        features.append(tempo)
        features.append(track_name)
        features.reverse()
        features.append(artist)
        newrow = np.transpose(features).reshape(1,-1)
        df = pd.DataFrame(newrow.reshape(1,-1),columns=cols).append(df, ignore_index=True)


06 - Live At Wembley Stadium London 1.mp3
02 - Wherever I May Roam.mp3
03 - Some Kind Of Monster.mp3
09 - Last Caress.mp3
05 - One.mp3
04 - Dirty Window.mp3
09 - Poor Twisted Me.mp3
11 - Low Man's Lyric.mp3
10 - Devil's Dance.mp3
05 - All Nightmare Long.mp3
15 - For Whom The Bell Tolls (Live).mp3
08 - The Judas Kiss.mp3
07 - Killing Time.mp3
10 - Purify.mp3
07 - Tuesday's Gone.mp3
10 - Prince Charming.mp3
05 - A) Last Caress; B) Green Hell.mp3
08 - Justice Medley.mp3
08 - One.mp3
10 - The Wait.mp3
12 - Attitude.mp3
05 - (Anesthesia) Pulling Teeth.mp3
03 - Sabbra Cadabra.mp3
08 - Nothing Else Matters.mp3
06 - Seek And Destroy.mp3
05 - Seek & Destroy.mp3
02 - The Call Of Ktulu.mp3
03 - Eye Of The Beholder.mp3
18 - Whiplash.mp3
10 - Last Caress.mp3
16 - Stone Dead Forever.mp3
06 - Harvester Of Sorrow.mp3
09 - Where The Wild Things Are.mp3
09 - Suicide & Redemption.mp3
17 - A) Prowler B) Run To The Hills.mp3
04 - Turn The Page.mp3
03 - For Whom The Bell Tolls.mp3
05 - Last Caress - Green H

In [7]:
df.tail()

,track_name,bpm,harmonic_rythmic_ratio,spectral_centroid,spectral_flatness,count_delta_above_mean,ratio_above_rmse_mean,max_rmse,mean_rms,mean_zcr,liked
6323,06 - Live At Wembley Stadium London 1.mp3,135.99917763157896,2.012115,2743.487061452844,0.041087333,0,0.8735177865612648,0.22269402,0.16670083,0.14009860396888185,Metallica
6324,06 - Live At Wembley Stadium London 1.mp3,129.19921875,2.053566,2396.664545732893,0.024036167,0,0.9190283400809717,0.23766279,0.17079656,0.11901185884757384,Metallica
6325,06 - Live At Wembley Stadium London 1.mp3,129.19921875,1.9386886,2346.376283673442,0.022524519,0,0.8809523809523809,0.23668507,0.17047647,0.11788901799841772,Metallica
6326,06 - Live At Wembley Stadium London 1.mp3,129.19921875,2.1696758,2262.935753709684,0.021709993,0,1.0256410256410255,0.23880228,0.17117251,0.10669151338343882,Metallica
6327,06 - Live At Wembley Stadium London 1.mp3,129.19921875,2.7291753,2969.3739130557583,0.06191542,0,0.6013513513513513,0.23448437,0.095827565,0.1599450735100211,Metallica


In [8]:
df.to_pickle('metallica.pkl')